In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
from sklearn.model_selection import train_test_split

In [3]:
train_set = pd.read_json('/Users/shidhesh/Downloads/train.json')

In [4]:
X_train, X_val = train_test_split(train_set,
                                  test_size=.2,
                                  shuffle=True,
                                  random_state=41,
                                  stratify= train_set['cuisine'])

In [5]:
#Set index in ascending order
X_val=X_val.set_index(np.arange(0,len(X_val)))
X_train=X_train.set_index(np.arange(0,len(X_train)))

In [6]:
#Number of unique ingredients in the test set
ing_col = X_train.ingredients.values
all_ings = np.concatenate(tuple(ing_col[i] for i in range(len(ing_col))))
unique_ings, counts = np.unique(all_ings, return_counts=True)
print('Unique number of ingredients in dataset: ', len(unique_ings))
ings=pd.DataFrame(all_ings, columns=["Ingredients"])
unique_in=ings.Ingredients.unique()

Unique number of ingredients in dataset:  6308


In [7]:
#Grouped by cuisines
count_by_cuisine = train_set.groupby(['cuisine']).count()

In [8]:
#Here we will construct a dataframe with ingredients as index, 
#cuisines as columns and fill it with number of occurences in X_train
temp=pd.DataFrame(0, index= unique_in, columns= count_by_cuisine.index.values)
for i in range(len(X_train)):
    for j in range(len(X_train.ingredients[i])):
        temp[X_train["cuisine"][i]][X_train.ingredients[i][j]]=temp[X_train["cuisine"][i]][X_train.ingredients[i][j]]+1

In [9]:
temp.head(5)

,brazilian,british,cajun_creole,chinese,filipino,french,greek,indian,irish,italian,jamaican,japanese,korean,mexican,moroccan,russian,southern_us,spanish,thai,vietnamese
vinaigrette dressing,0,0,1,0,0,0,3,0,0,3,0,0,1,0,0,0,0,0,0,0
ground black pepper,45,59,202,115,106,312,165,167,54,1135,69,55,69,453,121,36,374,122,36,85
cherry tomatoes,1,4,5,5,0,30,38,11,0,135,1,2,0,57,10,0,9,5,27,0
lemon juice,10,17,48,39,8,72,157,222,9,166,5,33,8,74,60,22,135,22,14,7
kale,7,1,4,3,0,2,2,6,3,49,5,8,3,11,3,1,9,8,5,1


In [11]:
#Next we will divide the rows by the total number of occurences of corresponding ingredients
temp['Total']=temp.sum(axis=1)
a=temp.sort_values(by=['Total'], ascending=False)
norm_temp =a.divide(a['Total'],axis=0)
norm_temp.drop(["Total"], axis=1, inplace=True)
norm_temp.head(5)

,brazilian,british,cajun_creole,chinese,filipino,french,greek,indian,irish,italian,jamaican,japanese,korean,mexican,moroccan,russian,southern_us,spanish,thai,vietnamese
salt,0.005378,0.010930,0.020576,0.025260,0.011659,0.033102,0.015857,0.054129,0.010375,0.096183,0.009056,0.011277,0.007217,0.075573,0.011450,0.007842,0.063393,0.013012,0.010652,0.007078
olive oil,0.007156,0.003617,0.018874,0.008179,0.003224,0.032951,0.031850,0.022255,0.003067,0.197153,0.004089,0.003460,0.002359,0.081472,0.025244,0.002988,0.019110,0.023435,0.007235,0.002281
onions,0.008281,0.008991,0.032571,0.018770,0.019953,0.023502,0.010883,0.074842,0.008517,0.076972,0.010883,0.009385,0.012934,0.093691,0.017350,0.008991,0.030126,0.015852,0.010726,0.006782
water,0.006166,0.007332,0.018497,0.051908,0.020663,0.028995,0.009582,0.055074,0.006832,0.071071,0.010165,0.026162,0.015914,0.061240,0.012331,0.006499,0.045659,0.010915,0.020497,0.014498
garlic,0.005741,0.002448,0.024485,0.052094,0.022796,0.014775,0.014522,0.049054,0.003124,0.099797,0.009625,0.010976,0.021023,0.089328,0.009794,0.001013,0.016380,0.008274,0.029298,0.015451


In [13]:
#Next we will construct similar dataframe which is further normalized by the occurence of cuisines
#This will provide a vector of 20 dimension for each ingredient
n_norm_temp=norm_temp.copy()
count_by_cuisine_train = X_train.groupby(['cuisine']).count()
n_norm_temp=n_norm_temp.divide(count_by_cuisine_train.id.values, axis='columns')
n_norm_temp.head(5)

,brazilian,british,cajun_creole,chinese,filipino,french,greek,indian,irish,italian,jamaican,japanese,korean,mexican,moroccan,russian,southern_us,spanish,thai,vietnamese
salt,0.000014,0.000017,0.000017,0.000012,0.000019,0.000016,0.000017,0.000023,0.000019,0.000015,0.000022,0.000010,0.000011,0.000015,0.000017,0.000020,0.000018,0.000016,0.000009,0.000011
olive oil,0.000019,0.000006,0.000015,0.000004,0.000005,0.000016,0.000034,0.000009,0.000006,0.000031,0.000010,0.000003,0.000004,0.000016,0.000038,0.000008,0.000006,0.000030,0.000006,0.000003
onions,0.000022,0.000014,0.000026,0.000009,0.000033,0.000011,0.000012,0.000031,0.000016,0.000012,0.000026,0.000008,0.000019,0.000018,0.000026,0.000023,0.000009,0.000020,0.000009,0.000010
water,0.000016,0.000011,0.000015,0.000024,0.000034,0.000014,0.000010,0.000023,0.000013,0.000011,0.000024,0.000023,0.000024,0.000012,0.000019,0.000017,0.000013,0.000014,0.000017,0.000022
garlic,0.000015,0.000004,0.000020,0.000024,0.000038,0.000007,0.000015,0.000020,0.000006,0.000016,0.000023,0.000010,0.000032,0.000017,0.000015,0.000003,0.000005,0.000010,0.000024,0.000023


In [14]:
#Predict the cuisine training data set
out_train=[0]*len(X_train)


for i in range(len(X_train)):
    s=[0]*20
    for j in range(len(X_train.ingredients[i])):
        s =s+n_norm_temp.loc[X_train.ingredients[i][j]]
    out_train[i]=s.argmax()

In [15]:
#Convert the actual training data output in numbers (0,20)
tr_set=X_train.copy()

y_true=tr_set.replace(to_replace=count_by_cuisine.index, value=np.arange(0,20,1)).cuisine.values

In [16]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, f1_score, recall_score, classification_report


In [17]:
#Build confusion matrix for training data
n_con_mat=confusion_matrix(y_true, out_train)
n_c_mat=pd.DataFrame(n_con_mat, index= count_by_cuisine.index.values, columns= count_by_cuisine.index.values)
n_c_mat

,brazilian,british,cajun_creole,chinese,filipino,french,greek,indian,irish,italian,jamaican,japanese,korean,mexican,moroccan,russian,southern_us,spanish,thai,vietnamese
brazilian,270,4,28,1,6,3,0,7,0,1,10,0,0,3,7,4,4,6,18,2
british,10,501,6,1,2,6,2,8,40,0,19,1,0,1,6,29,9,0,0,2
cajun_creole,7,18,1072,0,1,8,3,4,4,4,14,0,3,3,10,48,32,4,0,2
chinese,11,28,16,1673,24,4,1,7,2,1,18,38,177,1,10,12,4,2,37,72
filipino,15,5,4,29,434,2,2,5,5,2,11,3,16,1,3,9,3,4,21,30
french,30,179,70,5,15,1160,78,6,88,47,61,7,13,3,81,121,42,96,7,8
greek,3,16,14,0,5,7,749,8,4,4,11,0,4,0,78,24,3,7,0,3
indian,19,26,12,3,4,1,31,1962,19,0,31,0,5,0,190,26,3,2,53,15
irish,6,39,7,3,3,10,6,2,398,1,13,0,1,0,15,22,5,1,2,0
italian,56,243,351,17,38,356,453,17,132,3505,99,12,43,34,271,219,80,297,14,33


In [18]:
# Training set accuracy
acc=0
for i in n_c_mat.index.values:
    acc=acc+n_c_mat[i][i]

In [19]:
acc_train=acc/(n_c_mat.sum().sum())
acc_train

0.6866652000377134

In [20]:
print("Precision score for training set is ", precision_score(y_true, out_train, average="macro"))
print("f1 score for training set is ",f1_score(y_true, out_train, average="macro"))
print("Recall score for training set is ",recall_score(y_true, out_train, average="macro")) 

Precision score for training set is  0.6112784186575013
f1 score for training set is  0.6363978180696894
Recall score for training set is  0.7436696300097323


In [21]:
print(classification_report(y_true, out_train, target_names= count_by_cuisine.index.values))

              precision    recall  f1-score   support

   brazilian       0.37      0.72      0.49       374
     british       0.35      0.78      0.48       643
cajun_creole       0.44      0.87      0.59      1237
     chinese       0.88      0.78      0.83      2138
    filipino       0.63      0.72      0.67       604
      french       0.68      0.55      0.61      2117
       greek       0.52      0.80      0.63       940
      indian       0.87      0.82      0.84      2402
       irish       0.43      0.75      0.55       534
     italian       0.97      0.56      0.71      6270
    jamaican       0.37      0.89      0.52       421
    japanese       0.91      0.72      0.80      1139
      korean       0.59      0.90      0.71       664
     mexican       0.97      0.76      0.85      5150
    moroccan       0.37      0.91      0.53       657
     russian       0.29      0.78      0.42       391
 southern_us       0.86      0.47      0.61      3456
     spanish       0.45    

In [22]:
X_val.head(5)

,id,cuisine,ingredients
0,13751,thai,"[fish sauce, water chestnuts, purple onion, se..."
1,12652,chinese,"[ground ginger, dry roasted peanuts, dark sesa..."
2,30957,mexican,"[cheddar cheese, fresh cilantro, chili powder,..."
3,3007,mexican,"[black beans, non-fat sour cream, chopped cila..."
4,29422,southern_us,"[cider vinegar, tupelo honey, olive oil, beets..."


In [24]:
#Predicting on the validation data
out_val=[0]*len(X_val)


for i in range(len(X_val)):
    s = ([0]*20)
    for j in range(len(X_val.ingredients[i])):
        if n_norm_temp.index.isin([X_val.ingredients[i][j]]).any()==True:
            s =s+n_norm_temp.loc[X_val.ingredients[i][j]]
    out_val[i]=pd.Series(s).argmax() 

In [25]:
#Convert the validation data set output in numbers (0,20)
v_set=X_val.copy()

y_true=v_set.replace(to_replace=count_by_cuisine.index, value=np.arange(0,20,1)).cuisine.values

In [26]:
#Build confusion matrix for validation data
n_con_val_mat=confusion_matrix(y_true, out_val)
n_c_val_mat=pd.DataFrame(n_con_val_mat, index= count_by_cuisine.index.values, columns= count_by_cuisine.index.values)
n_c_val_mat

,brazilian,british,cajun_creole,chinese,filipino,french,greek,indian,irish,italian,jamaican,japanese,korean,mexican,moroccan,russian,southern_us,spanish,thai,vietnamese
brazilian,44,3,9,1,2,0,1,0,2,0,6,0,1,1,2,4,3,4,9,1
british,6,84,1,0,4,3,0,4,16,0,11,0,1,0,8,13,9,1,0,0
cajun_creole,3,7,257,0,1,3,3,2,2,4,5,0,0,1,2,6,9,2,0,2
chinese,2,8,5,386,8,1,0,1,1,1,4,7,61,1,3,3,1,1,10,31
filipino,8,3,4,11,91,0,0,0,1,0,3,3,7,1,1,2,0,0,6,10
french,5,87,22,1,5,237,18,2,16,16,16,2,1,0,27,39,8,24,1,2
greek,0,9,2,0,2,2,170,1,3,4,6,0,1,0,21,7,0,5,1,1
indian,7,3,3,0,1,0,14,469,4,0,14,0,1,0,51,10,0,2,16,6
irish,4,23,2,0,1,2,1,0,75,0,4,0,0,1,8,7,4,0,1,0
italian,15,59,91,8,10,83,128,4,34,834,35,5,16,7,76,51,14,87,5,6


In [27]:
# Validation set accuracy
acc=0
for i in n_c_val_mat.index.values:
    acc=acc+n_c_val_mat[i][i]

In [28]:
acc_val=acc/(n_c_val_mat.sum().sum())
acc_val

0.6290383406662476

In [29]:
print("Precision score for test set is ", precision_score(y_true, out_val, average="macro"))
print("f1 score for test set is ",f1_score(y_true, out_val, average="macro"))
print("Recall score for test set is ",recall_score(y_true, out_val, average="macro")) 

Precision score for test set is  0.5500735808652528
f1 score for test set is  0.5619481054483884
Recall score for test set is  0.6593421781960019


In [30]:
print(classification_report(y_true, out_val, target_names= count_by_cuisine.index.values))

              precision    recall  f1-score   support

   brazilian       0.24      0.47      0.32        93
     british       0.21      0.52      0.30       161
cajun_creole       0.42      0.83      0.56       309
     chinese       0.84      0.72      0.78       535
    filipino       0.50      0.60      0.55       151
      french       0.64      0.45      0.53       529
       greek       0.44      0.72      0.55       235
      indian       0.88      0.78      0.82       601
       irish       0.34      0.56      0.42       133
     italian       0.95      0.53      0.68      1568
    jamaican       0.30      0.87      0.44       105
    japanese       0.85      0.69      0.76       284
      korean       0.52      0.89      0.65       166
     mexican       0.97      0.73      0.83      1288
    moroccan       0.34      0.90      0.49       164
     russian       0.23      0.62      0.34        98
 southern_us       0.82      0.41      0.54       864
     spanish       0.35    